Test Data

In [ ]:
import pandas as pd

# Load the Excel file
xls = pd.ExcelFile('/content/Income&Expensemaster.xlsx')

# Load specific sheets
income_expense = pd.read_excel(xls, 'Income&Expense')
balance_sheet = pd.read_excel(xls, 'BalanceSheet')

# Print column names to verify
print("Income & Expense Columns:", income_expense.columns)
print("Balance Sheet Columns:", balance_sheet.columns)


Income & Expense Columns: Index(['Particulars',          2023,          2022,          2021,
                2020,          2019,          2018,          2017,
                2016,          2015,          2014,          2013,
                2012,          2011,          2010,          2009,
                2008,          2007],
      dtype='object')
Balance Sheet Columns: Index(['Particulars',          2023,          2022,          2021,
                2020,          2019,          2018,          2017,
                2016,          2015,          2014,          2013,
                2012,          2011,          2010,          2009,
                2008,          2007],
      dtype='object')


In [ ]:
pip install openpyxl

In [ ]:
import pandas as pd

def fill_na_with_column_mean(df):
    # Exclude 'Particulars' from mean calculation and fillna operation
    numeric_cols = df.columns.drop('Particulars')
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')  # Convert columns to numeric, coerce errors
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())  # Fill NA/NaN with mean of each column
    return df

# Load the data
income_expense = pd.read_excel('Income&Expensemaster.xlsx', sheet_name='Income&Expense')
balance_sheet = pd.read_excel('Income&Expensemaster.xlsx', sheet_name='BalanceSheet')

# Replace NA and NULL values with the mean of each column, excluding non-numeric columns
income_expense_cleaned = fill_na_with_column_mean(income_expense)
balance_sheet_cleaned = fill_na_with_column_mean(balance_sheet)

# Save the cleaned data back to an Excel file
with pd.ExcelWriter('Processed_CreditSuisse_Data.xlsx') as writer:
    income_expense_cleaned.to_excel(writer, sheet_name='Income&Expense', index=False)
    balance_sheet_cleaned.to_excel(writer, sheet_name='BalanceSheet', index=False)

print("Data processing complete and saved to 'Processed_CreditSuisse_Data.xlsx'.")



Data processing complete and saved to 'Processed_CreditSuisse_Data.xlsx'.


Current Assets

In [ ]:
import pandas as pd

# Load the balance sheet data from the specific file and sheet
balance_sheet = pd.read_excel('/content/Processed_CreditSuisse_Data.xlsx', sheet_name='BalanceSheet')

# Define the rows that contribute to current assets
current_assets_items = [
    "Cash and other liquid assets",
    "Money market instruments",
    "Due from banks",
    "Securities and precious metals trading portfolio",
    "Due from customers",
    "Trading assets",
    "Positive replacement values of derivative financial instruments",
    "Other financial instruments held at fair value",
    "Accrued income and prepaid expenses"
]

# Filter the balance sheet to include only the rows for current assets
current_assets_data = balance_sheet[balance_sheet['Particulars'].isin(current_assets_items)]

# Since the data might have years as columns, sum across the rows for each year to get total current assets
current_assets_annual = current_assets_data.iloc[:, 1:].sum()  # Skip the 'Particulars' column

# Convert the series to a DataFrame for easier manipulation and to properly define column names
current_assets_df = pd.DataFrame(current_assets_annual, columns=['current_assets']).reset_index()
current_assets_df.rename(columns={'index': 'Year'}, inplace=True)

# Save to Excel if needed
current_assets_df.to_excel('Annual_Current_Assets.xlsx', index=False)

print("Current assets calculation completed and saved.")


Current assets calculation completed and saved.


Cost of goods sold- Direct expense as proxy

In [ ]:
import pandas as pd

# Load the Income & Expense data
income_expense = pd.read_excel('Processed_CreditSuisse_Data.xlsx', sheet_name='Income&Expense')

# Define the rows that contribute to the cost of goods sold (using proxies as discussed)
cogs_items = [
    "Interest expense",
    "Commission expense"
]

# Filter the data to include only the rows for cost of goods sold
cogs_data = income_expense[income_expense['Particulars'].isin(cogs_items)]

# Sum across the rows for each year to get total cost of goods sold
cogs_annual = cogs_data.iloc[:, 1:].sum()  # Skip the 'Particulars' column

# Convert the series to a DataFrame for easier manipulation and to properly define column names
cogs_df = pd.DataFrame(cogs_annual, columns=['cost_of_goods_sold']).reset_index()
cogs_df.rename(columns={'index': 'Year'}, inplace=True)

# Save to Excel
cogs_df.to_excel('Annual_COGS.xlsx', index=False)

print("Cost of Goods Sold calculation completed and saved.")


Cost of Goods Sold calculation completed and saved.


depriciation

In [ ]:
import pandas as pd

# Load the data
income_expense = pd.read_excel('Processed_CreditSuisse_Data.xlsx', sheet_name='Income&Expense')

# Define the row for depreciation and amortization
dep_amort_item = "Impairment of participations, depreciation and amortization of tangible fixed assets and intangible assets"

# Select the row with depreciation and amortization data
dep_amort_data = income_expense[income_expense['Particulars'] == dep_amort_item]

# Extract only the yearly data and convert it to a DataFrame
dep_amort_annual = dep_amort_data.iloc[0, 1:]  # skip 'Particulars' column
dep_amort_df = pd.DataFrame(dep_amort_annual)

# Transpose to get years as rows
dep_amort_df = dep_amort_df.transpose()

# Reset the index to make 'Year' a column
dep_amort_df.reset_index(inplace=True)

# Print the structure to check it before renaming columns
print(dep_amort_df.head())
print(dep_amort_df.columns)

# If the structure is correct and there are indeed two columns, set the column names
if len(dep_amort_df.columns) == 2:
    dep_amort_df.columns = ['Year', 'depreciation_amortization']
    print("Columns renamed successfully.")
else:
    print("Column mismatch error: expected 2 columns, found", len(dep_amort_df.columns))

# Optionally, convert the 'Year' from column names to a proper datetime format if necessary
# dep_amort_df['Year'] = pd.to_datetime(dep_amort_df['Year'], format='%Y')

# Save to Excel
dep_amort_df.to_excel('Depreciation_and_Amortization_Annual.xlsx', index=False)

print("Depreciation and Amortization data is processed and saved.")


   index  2023   2022   2021  2020   2019  2018 2017  2016  2015  2014  \
0     24  3571  12379  12884  4834  18288  2126  432  1459  5401  4517   

     2013    2012   2011    2010   2009  2008    2007  
0  2894.0  3267.0  602.0  5444.0  661.0  4753  1251.0  
Index(['index',    2023,    2022,    2021,    2020,    2019,    2018,    2017,
          2016,    2015,    2014,    2013,    2012,    2011,    2010,    2009,
          2008,    2007],
      dtype='object')
Column mismatch error: expected 2 columns, found 18
Depreciation and Amortization data is processed and saved.


In [ ]:
import pandas as pd

# Load the processed data
income_expense_path = "Processed_CreditSuisse_Data.xlsx"
income_sheet = pd.read_excel(income_expense_path, sheet_name='Income&Expense', index_col=0)

# We'll calculate EBITDA as:
# EBITDA = Net Operating Income + Interest Expense + Taxes + Depreciation & Amortization

# Extract necessary rows
net_operating_income = income_sheet.loc['Net Operating income']
interest_expense = -income_sheet.loc['Interest expense']  # Negate since it's an expense
taxes = -income_sheet.loc['Taxes']  # Negate since it's an expense
depreciation_amortization = income_sheet.loc['Impairment of participations, depreciation and amortization of tangible fixed assets and intangible assets']

# Combine all components
ebitda = net_operating_income + interest_expense + taxes + depreciation_amortization

# Create DataFrame for easier handling and visualization
ebitda_df = pd.DataFrame(ebitda, columns=['EBITDA'])
ebitda_df.reset_index(inplace=True)
ebitda_df.rename(columns={'index': 'Year'}, inplace=True)

# Save the final EBITDA calculations to an Excel file
ebitda_df.to_excel("EBITDA_Credit_Suisse.xlsx", index=False)

print("EBITDA calculation completed and saved.")


EBITDA calculation completed and saved.


In [ ]:
import pandas as pd

# Load the data
df_balance_sheet = pd.read_excel('Processed_CreditSuisse_Data.xlsx', sheet_name='BalanceSheet')

# Columns representing the years
year_columns = df_balance_sheet.columns[1:]  # Adjust based on actual column indexing

# Extracting 'Trading assets' as a proxy for Inventory
inventory_proxy = df_balance_sheet[df_balance_sheet['Particulars'] == 'Trading assets'][year_columns].transpose()
inventory_proxy.columns = ['Inventory']

# Reset index to turn the years into a column
inventory_proxy.reset_index(inplace=True)
inventory_proxy.rename(columns={'index': 'Year'}, inplace=True)

# Optionally, save this data
inventory_proxy.to_excel('Inventory_Proxy_Data.xlsx', index=False)

print(inventory_proxy)


    Year      Inventory
0   2023    7217.000000
1   2022   26072.000000
2   2021   39410.000000
3   2020   49116.000000
4   2019   51640.000000
5   2018   42781.000000
6   2017   48629.000000
7   2016   17586.000000
8   2015   31710.000000
9   2014  107299.083333
10  2013   96029.722222
11  2012   95584.228571
12  2011   99660.457143
13  2010  109061.875000
14  2009  106301.750000
15  2008  122207.000000
16  2007  126601.062500


In [ ]:
import pandas as pd

# Load the data
df_income_expense = pd.read_excel('Processed_CreditSuisse_Data.xlsx', sheet_name='Income&Expense')

# Columns representing the years
year_columns = df_income_expense.columns[1:]  # Adjust based on actual column indexing

# Extracting 'Net profit/(loss)' as 'Net income'
net_income = df_income_expense[df_income_expense['Particulars'] == 'Net profit/(loss)'][year_columns].transpose()
net_income.columns = ['Net Income']

# Reset index to turn the years into a column
net_income.reset_index(inplace=True)
net_income.rename(columns={'index': 'Year'}, inplace=True)

# Optionally, save this data
net_income.to_excel('Net_Income_Data.xlsx', index=False)

print(net_income)


    Year  Net Income
0   2023     10126.0
1   2022    -12565.0
2   2021    -11009.0
3   2020       331.0
4   2019    -11385.0
5   2018      -647.0
6   2017       225.0
7   2016     -2837.0
8   2015     -2877.0
9   2014     -3092.0
10  2013      1066.0
11  2012       183.0
12  2011      1276.0
13  2010     -2674.0
14  2009       378.0
15  2008     -2240.0
16  2007      3625.0


In [ ]:
import pandas as pd

# Load the data
df_balance_sheet = pd.read_excel('Processed_CreditSuisse_Data.xlsx', sheet_name='BalanceSheet')

# Columns representing the years
year_columns = df_balance_sheet.columns[1:]  # Adjust based on actual column indexing

# Extracting 'Due from customers' as 'Total Receivables'
total_receivables = df_balance_sheet[df_balance_sheet['Particulars'] == 'Due from customers'][year_columns].transpose()
total_receivables.columns = ['Total Receivables']

# Reset index to turn the years into a column
total_receivables.reset_index(inplace=True)
total_receivables.rename(columns={'index': 'Year'}, inplace=True)

# Optionally, save this data
total_receivables.to_excel('Total_Receivables_Data.xlsx', index=False)

print(total_receivables)


    Year  Total Receivables
0   2023            67628.0
1   2022           117543.0
2   2021           153874.0
3   2020           172051.0
4   2019           185713.0
5   2018           177104.0
6   2017           193106.0
7   2016           180426.0
8   2015           241190.0
9   2014           233422.0
10  2013           192376.0
11  2012           188085.0
12  2011           167712.0
13  2010           173743.0
14  2009           198696.0
15  2008           201661.0
16  2007           241880.0


In [ ]:
import pandas as pd

# Load the data
df_balance_sheet = pd.read_excel('Processed_CreditSuisse_Data.xlsx', sheet_name='BalanceSheet')

# Columns representing the years
year_columns = df_balance_sheet.columns[1:]  # Adjust based on actual column indexing

# Extracting 'Total Assets' as 'Market Value'
market_value = df_balance_sheet[df_balance_sheet['Particulars'] == 'Total assets'][year_columns].transpose()
market_value.columns = ['Market Value']

# Reset index to turn the years into a column
market_value.reset_index(inplace=True)
market_value.rename(columns={'index': 'Year'}, inplace=True)

# Optionally, save this data
market_value.to_excel('Market_Value_Data.xlsx', index=False)

print(market_value)


    Year  Market Value
0   2023      257935.0
1   2022      378363.0
2   2021      539213.0
3   2020      548675.0
4   2019      569296.0
5   2018      532816.0
6   2017      576218.0
7   2016      507800.0
8   2015      662010.0
9   2014      682347.0
10  2013      606362.0
11  2012      589063.0
12  2011      620611.0
13  2010      618652.0
14  2009      597255.0
15  2008      686901.0
16  2007      715602.0


In [ ]:
import pandas as pd

# Load the data
df_income_expense = pd.read_excel('Processed_CreditSuisse_Data.xlsx', sheet_name='Income&Expense')

# Columns representing the years
year_columns = df_income_expense.columns[1:]  # Adjust based on actual column indexing

# Define the parts that make up 'Net Sales'
components = [
    "Net income from commission and service activities",
    "Net income from interest activities",
    "Net income/(loss) from trading activities and fair value option"
]

# Extract the relevant rows and sum them to approximate 'Net Sales'
net_sales = df_income_expense[df_income_expense['Particulars'].isin(components)][year_columns].sum().to_frame()
net_sales.columns = ['Net Sales']

# Reset index to turn the years into a column
net_sales.reset_index(inplace=True)
net_sales.rename(columns={'index': 'Year'}, inplace=True)

# Optionally, save this data
net_sales.to_excel('Net_Sales_Data.xlsx', index=False)

print(net_sales)


    Year  Net Sales
0   2023    -1079.0
1   2022     1492.0
2   2021     4100.0
3   2020     6458.0
4   2019     4816.0
5   2018     4126.0
6   2017     4340.0
7   2016     4352.0
8   2015     9669.0
9   2014    10079.0
10  2013     9666.0
11  2012     8192.0
12  2011     8402.0
13  2010     8515.0
14  2009     7509.0
15  2008     8664.0
16  2007    12004.0


In [ ]:
import pandas as pd

# Load the data
df_balance_sheet = pd.read_excel('Processed_CreditSuisse_Data.xlsx', sheet_name='BalanceSheet')

# Columns representing the years
year_columns = df_balance_sheet.columns[1:]  # Adjust based on actual column indexing

# Extract the row for 'Total assets'
total_assets = df_balance_sheet[df_balance_sheet['Particulars'] == "Total assets"][year_columns].transpose()
total_assets.columns = ['Total Assets']

# Reset index to turn the years into a column
total_assets.reset_index(inplace=True)
total_assets.rename(columns={'index': 'Year'}, inplace=True)

# Optionally, save this data
total_assets.to_excel('Total_Assets_Data.xlsx', index=False)

print(total_assets)


    Year  Total Assets
0   2023      257935.0
1   2022      378363.0
2   2021      539213.0
3   2020      548675.0
4   2019      569296.0
5   2018      532816.0
6   2017      576218.0
7   2016      507800.0
8   2015      662010.0
9   2014      682347.0
10  2013      606362.0
11  2012      589063.0
12  2011      620611.0
13  2010      618652.0
14  2009      597255.0
15  2008      686901.0
16  2007      715602.0


In [ ]:
import pandas as pd

# Load the data
df_balance_sheet = pd.read_excel('Processed_CreditSuisse_Data.xlsx', sheet_name='BalanceSheet')

# Columns representing the years
year_columns = df_balance_sheet.columns[1:]  # Adjust based on actual column indexing

# Identify and sum relevant rows for long-term debt
# Adjust the rows being summed based on actual data and definitions in your dataset
long_term_debt_items = ['Bonds and mortgage-backed bonds']

# Filter rows and sum the relevant data
total_long_term_debt = df_balance_sheet[df_balance_sheet['Particulars'].isin(long_term_debt_items)].loc[:, year_columns].sum()

# Convert the series to a DataFrame for easier manipulation and to properly define column names
total_long_term_debt_df = pd.DataFrame(total_long_term_debt, columns=['Total Long Term Debt'])

# Reset index to turn the years into a column
total_long_term_debt_df.reset_index(inplace=True)
total_long_term_debt_df.rename(columns={'index': 'Year'}, inplace=True)

# Optionally, save this data
total_long_term_debt_df.to_excel('Total_Long_Term_Debt_Data.xlsx', index=False)

print(total_long_term_debt_df)


    Year  Total Long Term Debt
0   2023               69892.0
1   2022              111770.0
2   2021              118959.0
3   2020              129446.0
4   2019              154790.0
5   2018              121793.0
6   2017              149831.0
7   2016              164958.0
8   2015              159134.0
9   2014              125697.0
10  2013               89348.0
11  2012              107573.0
12  2011              118613.0
13  2010              119051.0
14  2009              102426.0
15  2008               79617.0
16  2007               60127.0


In [ ]:
import pandas as pd

# Load the processed Excel file
df_income_expense = pd.read_excel('Processed_CreditSuisse_Data.xlsx', sheet_name='Income&Expense')

# Identify the rows for Gross Operating Profit and Total Operating Expenses
gross_profit_row = df_income_expense[df_income_expense['Particulars'] == 'Gross operating profit']
operating_expenses_row = df_income_expense[df_income_expense['Particulars'] == 'Total operating expenses']

# Calculate EBIT for each year
ebit = gross_profit_row.iloc[:, 1:].values - operating_expenses_row.iloc[:, 1:].values
ebit_df = pd.DataFrame(ebit, columns=gross_profit_row.columns[1:])

# Transpose the DataFrame for easier manipulation and saving
ebit_df = ebit_df.transpose()
ebit_df.columns = ['EBIT']
ebit_df['Year'] = ebit_df.index

# Reset index and optionally save the result
ebit_df.reset_index(drop=True, inplace=True)
ebit_df.to_excel('EBIT_Credit_Suisse_2007_to_2023.xlsx', index=False)

print(ebit_df)


      EBIT  Year
0  -7090.0  2023
1  -6370.0  2022
2  -4322.0  2021
3  -1476.0  2020
4  -3921.0  2019
5  -3999.0  2018
6  -4761.0  2017
7  -8093.0  2016
8  -4134.0  2015
9  -2885.0  2014
10 -2165.0  2013
11 -3985.0  2012
12 -4345.0  2011
13 -4294.0  2010
14 -5991.0  2009
15 -1649.0  2008
16  -962.0  2007


In [ ]:
import pandas as pd

# Load the processed Excel file
df_income_expense = pd.read_excel('Processed_CreditSuisse_Data.xlsx', sheet_name='Income&Expense')

# Extract the row corresponding to Gross Operating Profit
gross_profit_row = df_income_expense[df_income_expense['Particulars'] == 'Gross operating profit']

# Since the values start from the second column, we exclude the 'Particulars' column
gross_profit = gross_profit_row.iloc[:, 1:]

# Transpose the result to have years as rows
gross_profit_df = gross_profit.transpose()
gross_profit_df.columns = ['Gross Profit']  # Rename the column to 'Gross Profit'
gross_profit_df['Year'] = gross_profit_df.index  # Add a 'Year' column based on the DataFrame's index

# Reset index to make 'Year' a proper column and not an index
gross_profit_df.reset_index(drop=True, inplace=True)

# Optionally, you might want to save the DataFrame to an Excel file
gross_profit_df.to_excel('Gross_Profit_Credit_Suisse_2007_to_2023.xlsx', index=False)

print(gross_profit_df)


    Gross Profit  Year
0        -1462.0  2023
1         -322.0  2022
2         1821.0  2021
3         4959.0  2020
4         2344.0  2019
5         1776.0  2018
6          857.0  2017
7        -1411.0  2016
8         3163.0  2015
9         3840.0  2014
10        4476.0  2013
11        3125.0  2012
12        2561.0  2011
13        2663.0  2010
14        1342.0  2009
15        3763.0  2008
16        5911.0  2007


In [ ]:
import pandas as pd

# Load the processed Excel file
df_balance_sheet = pd.read_excel('Processed_CreditSuisse_Data.xlsx', sheet_name='BalanceSheet')

# Define the items that are considered current liabilities
current_liabilities_items = [
    'Due to banks',
    'Securities lending and repurchase agreements',
    'Due to customers, savings and investment deposits',
    'Due to customers, other deposits',
    'Medium-term notes',
    'Trading liabilities',
    'Negative replacement values of derivative financial instruments',
    'Liabilities from other financial instruments held at fair value',
    'Accrued expenses and deferred income',
    'Other liabilities'
]

# Filter the data for these items
current_liabilities_rows = df_balance_sheet[df_balance_sheet['Particulars'].isin(current_liabilities_items)]

# Sum the values across these rows for each year
current_liabilities = current_liabilities_rows.iloc[:, 1:].sum()  # skip the 'Particulars' column

# Convert the series to a DataFrame for easier manipulation and to properly define column names
current_liabilities_df = pd.DataFrame(current_liabilities, columns=['Total Current Liabilities'])
current_liabilities_df['Year'] = current_liabilities_df.index  # Add a 'Year' column based on the DataFrame's index

# Reset index to make 'Year' a proper column and not an index
current_liabilities_df.reset_index(drop=True, inplace=True)

# Optionally, you might want to save the DataFrame to an Excel file
current_liabilities_df.to_excel('Total_Current_Liabilities_Credit_Suisse_2007_to_2023.xlsx', index=False)

print(current_liabilities_df)


    Total Current Liabilities  Year
0                2.334544e+05  2023
1                3.543943e+05  2022
2                5.394839e+05  2021
3                5.327890e+05  2020
4                5.335792e+05  2019
5                5.027527e+05  2018
6                5.249006e+05  2017
7                4.290497e+05  2016
8                6.236451e+05  2015
9                8.646803e+05  2014
10               8.007059e+05  2013
11               7.885309e+05  2012
12               8.125028e+05  2011
13               8.670845e+05  2010
14               8.691480e+05  2009
15               1.019967e+06  2008
16               1.060554e+06  2007


In [ ]:
import pandas as pd

# Load the processed Excel file
df_balance_sheet = pd.read_excel('Processed_CreditSuisse_Data.xlsx', sheet_name='BalanceSheet')

# Define the item that represents retained earnings
retained_earnings_item = "Retained earnings carried forward"

# Filter the data for this item
retained_earnings_row = df_balance_sheet[df_balance_sheet['Particulars'] == retained_earnings_item]

# Select the columns for years 2007 to 2023 and transpose the data
retained_earnings = retained_earnings_row.iloc[:, 1:].transpose()  # This assumes years are from column 2 onwards
retained_earnings.columns = ['Retained Earnings']  # Rename the column after transposing

# Reset index to turn the years into a column and correct the DataFrame structure
retained_earnings.reset_index(inplace=True)
retained_earnings.columns = ['Year', 'Retained Earnings']

# Optionally, save the DataFrame to an Excel file
retained_earnings.to_excel('Retained_Earnings_Credit_Suisse_2007_to_2023.xlsx', index=False)

print(retained_earnings)


    Year  Retained Earnings
0   2023           -30435.0
1   2022           -17870.0
2   2021           -10886.0
3   2020           -11217.0
4   2019              168.0
5   2018              215.0
6   2017                0.0
7   2016              226.0
8   2015             3113.0
9   2014             6215.0
10  2013             5159.0
11  2012             4986.0
12  2011             3720.0
13  2010             6404.0
14  2009             9304.0
15  2008            11554.0
16  2007            10768.0


In [ ]:
import pandas as pd

# Load the processed Excel file
df_income_expense = pd.read_excel('Processed_CreditSuisse_Data.xlsx', sheet_name='Income&Expense')

# Define the items that represent revenue components
revenue_items = [
    "Interest and discount income",
    "Interest and dividend income from trading activities",
    "Interest and dividend income from financial investments",
    "Commission income from securities trading and investment activities",
    "Commission income from lending activities",
    "Commission income from other services",
    "Net income/(loss) from trading activities and fair value option",
    "Income from participations",
    "Income from real estate",
    "Other ordinary income"
]

# Filter the data for these items and sum their values to compute total revenue
total_revenue = df_income_expense[df_income_expense['Particulars'].isin(revenue_items)].iloc[:, 1:].sum()

# Convert the series to a DataFrame for easier manipulation and to properly define column names
total_revenue_df = pd.DataFrame(total_revenue, columns=['Total Revenue']).transpose()

# Reset index to turn the years into a column, if you want to make 'Year' a separate column
total_revenue_df.reset_index(inplace=True)
total_revenue_df.rename(columns={'index': 'Year'}, inplace=True)

# Optionally, save the DataFrame to an Excel file
total_revenue_df.to_excel('Total_Revenue_Credit_Suisse_2007_to_2023.xlsx', index=False)

print(total_revenue_df)


            Year     2023     2022     2021     2020     2019     2018  \
0  Total Revenue  18979.0  14663.0  12962.0  19039.0  20554.0  18644.0   

      2017     2016     2015     2014     2013     2012     2011     2010  \
0  14324.0  12175.0  16261.0  16946.0  17575.0  17269.0  16999.0  17554.0   

      2009     2008     2007  
0  18021.0  29606.0  39018.0  


In [ ]:
import pandas as pd

# Load the Balance Sheet data
df_balance_sheet = pd.read_excel("Processed_CreditSuisse_Data.xlsx", sheet_name='BalanceSheet')

# Define year columns dynamically from the data
year_columns = df_balance_sheet.columns[1:]

# Define the rows that contribute to total liabilities
liability_items = [
    "Total liabilities",
]

# Calculate total liabilities
total_liabilities = df_balance_sheet[df_balance_sheet['Particulars'].isin(liability_items)].loc[:, year_columns].sum()
total_liabilities_df = pd.DataFrame(total_liabilities).transpose()
total_liabilities_df.columns = year_columns  # Ensure column names match the years

# Reset index to turn the years into a column, which will be more meaningful
total_liabilities_df.reset_index(inplace=True)
total_liabilities_df.rename(columns={'index': 'Year'}, inplace=True)

# Optionally, add a column name for the total values if only one column of data is expected
if total_liabilities_df.shape[1] == 2:
    total_liabilities_df.columns = ['Year', 'Total Liabilities']

# Save to Excel
total_liabilities_df.to_excel('Total_Liabilities.xlsx', index=False)

print("Total liabilities calculation is completed and saved.")


Total liabilities calculation is completed and saved.


In [ ]:
import pandas as pd

# Load the Income & Expense data
df_income_expense = pd.read_excel("Processed_CreditSuisse_Data.xlsx", sheet_name='Income&Expense')

# Define year columns dynamically from the data
year_columns = df_income_expense.columns[1:]

# Define the rows that contribute to total operating expenses
operating_expense_items = [
    "Total operating expenses",
]

# Calculate total operating expenses
total_operating_expenses = df_income_expense[df_income_expense['Particulars'].str.contains("Total operating expenses", case=False, na=False)].loc[:, year_columns].sum()

# Convert the series to a DataFrame for easier manipulation and to properly define column names
total_operating_expenses_df = pd.DataFrame(total_operating_expenses, columns=['Total Operating Expenses'])

# Reset index to turn the years into a column
total_operating_expenses_df.reset_index(inplace=True)
total_operating_expenses_df.rename(columns={'index': 'Year'}, inplace=True)

# Check the DataFrame structure
print(total_operating_expenses_df.head())

# Save to Excel
total_operating_expenses_df.to_excel('Total_Operating_Expenses.xlsx', index=False)

print("Total operating expenses calculation is completed and saved.")


   Year  Total Operating Expenses
0  2023                    5628.0
1  2022                    6048.0
2  2021                    6143.0
3  2020                    6435.0
4  2019                    6265.0
Total operating expenses calculation is completed and saved.


In [ ]:
import pandas as pd
import joblib

# Load the models
ada_boost_model = joblib.load('best_adaboost_model.joblib')
gradient_boosting_model = joblib.load('best_gradient_boosting_model.joblib')
knn_model = joblib.load('knn_model.joblib')
logistic_regression_model = joblib.load('logistic_regression_model.joblib')
random_forest_model = joblib.load('random_forest_model.joblib')

# Load your test data
df_test = pd.read_excel('/content/Test_Data.xlsx')

# Prepare the feature matrix by dropping non-feature columns
X_test = df_test.drop(['Status_Label', 'Company_name', 'Year'], axis=1)

# Predictions dictionary
predictions = {}

# Predict for each model and store results
for name, model in models.items():
    predictions[name] = model.predict(X_test)
    df_test[name + '_Predictions'] = predictions[name]  # Add predictions to the DataFrame

# Include 'Company_name' and 'Year' in the output for traceability
output_df = df_test[['Company_name', 'Year'] + [name + '_Predictions' for name in predictions]]

# Save the DataFrame with predictions back to an Excel file
output_df.to_excel('Predicted_Status_Labels.xlsx', index=False)

print("Predictions are completed and saved.")


Predictions are completed and saved.


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load the updated Excel file with predictions and actual labels
data = pd.read_excel('Predicted_Status_Labels.xlsx')

# Correct column name for actual labels
y_true = data['Actual_Labels']

# Dictionary to store evaluation results
evaluation_results = {}

# Dictionary of model predictions columns
models = {
    'AdaBoost': 'AdaBoost_Predictions',
    'Gradient Boosting': 'Gradient Boosting_Predictions',
    'KNN': 'KNN_Predictions',
    'Logistic Regression': 'Logistic Regression_Predictions',
    'Random Forest': 'Random Forest_Predictions'
}

# Evaluate each model
for name, pred_col in models.items():
    y_pred = data[pred_col]
    accuracy = accuracy_score(y_true, y_pred)
    confusion = confusion_matrix(y_true, y_pred)
    report = classification_report(y_true, y_pred, zero_division=0)

    evaluation_results[name] = {
        'Accuracy': accuracy,
        'Confusion Matrix': confusion,
        'Classification Report': report
    }

    print(f"Results for {name}:\n")
    print(f"Accuracy: {accuracy:.4f}\n")
    print("Confusion Matrix:")
    print(confusion)
    print("\nClassification Report:")
    print(report)
    print("\n" + "-"*50 + "\n")

# If needed, save the evaluation results to a new Excel file
pd.DataFrame(evaluation_results).to_excel('Model_Evaluation_Results.xlsx', index=False)


Results for AdaBoost:

Accuracy: 0.9412

Confusion Matrix:
[[16  0]
 [ 1  0]]

Classification Report:
              precision    recall  f1-score   support

       alive       0.94      1.00      0.97        16
      failed       0.00      0.00      0.00         1

    accuracy                           0.94        17
   macro avg       0.47      0.50      0.48        17
weighted avg       0.89      0.94      0.91        17


--------------------------------------------------

Results for Gradient Boosting:

Accuracy: 1.0000

Confusion Matrix:
[[16  0]
 [ 0  1]]

Classification Report:
              precision    recall  f1-score   support

       alive       1.00      1.00      1.00        16
      failed       1.00      1.00      1.00         1

    accuracy                           1.00        17
   macro avg       1.00      1.00      1.00        17
weighted avg       1.00      1.00      1.00        17


--------------------------------------------------

Results for KNN:

Accuracy: